# Capítulo 9: Editando e executando arquivos externos

Livro: Aprendendo Química com Python, Rodrigo Q. de Albuquerque & Gerd B. da Rocha, 2021, Amazon Book.

E-Mail: learn.chem.python@gmail.com

### Versão do Python usada

In [88]:
import sys
print(sys.version)

3.7.10 | packaged by conda-forge | (default, Feb 19 2021, 16:07:37) 
[GCC 9.3.0]


### Caixa de sessão interativa 38: Extraindo informação de arquivo

In [89]:
%%file file.txt
Total repulsion energy = 23.435238 a.u.
Total energy for this iteration = 22.694720 a.u.
Final geometry:
= = = = = = = = = = = = = = = = =
   X      Y      Z
H 0.0000 0.0000 0.0000
H 0.0000 0.0000 0.7400
Wall time: 1.321 seconds

Overwriting file.txt


In [90]:
with open('file.txt') as f:     # arquivo vai pro objeto 'f'
    f.readline()                # linha 1 é mostrada na tela
    f.readline()                # linha 2 é mostrada na tela

In [91]:
f

<_io.TextIOWrapper name='file.txt' mode='r' encoding='UTF-8'>

In [92]:
with open('file.txt') as f:
    line = f.readline()      # a linha 1 é salva como 'line'

In [93]:
line

'Total repulsion energy = 23.435238 a.u.\n'

In [94]:
line[:8]        # mostre os primeiros 8 caracteres de 'line'

'Total re'

In [95]:
line.split()                     # elimine '\n' e gere lista

['Total', 'repulsion', 'energy', '=', '23.435238', 'a.u.']

In [96]:
newline = line.split()

In [97]:
newline[4]          

'23.435238'

In [98]:
energy = float(newline[4])   # converta 'string' pra 'float'

In [99]:
energy / 10                   # faça operações com a energia

2.3435238

### Caixa de sessão interativa 39: Criando arquivos

In [100]:
with open("file.txt", 'w') as f:
    line = 'Info 1\n'     
    f.write(line)         # insira a primeira linha de texto
    line = 'Info 2' + '\n' 
    f.write(line)          # insira a segunda linha de texto
    f.write('Info 3\n')   # insira a terceira linha de texto

In [101]:
with open("file.txt") as f:
    f.readlines()

In [102]:
with open("file.txt") as f:
    myfile = f.read()

In [103]:
myfile

'Info 1\nInfo 2\nInfo 3\n'

### Caixa de código 6: Extraindo (dados de) e modiﬁcando arquivo

In [104]:
%%file results.txt
MC iteration 1, energy = 3.4232 hartree
MC iteration 2, energy = 3.4783 hartree
MC iteration 3, energy = 3.4155 hartree
MC iteration 4, energy = 3.4369 hartree
MC simulation will proceed for 2 more steps
MC iteration 5, energy = 3.4423 hartree
The MC iterations took ca 5.4321 seconds
MC iteration 6, energy = 3.4510 hartree

Overwriting results.txt


In [105]:
import numpy as np

energy = []                    # valor da energia é inicializado

with open('results.txt') as f:
    lines = f.readlines()  # 'lines' = lista com todas as linhas

for line in lines:                 # itere cada linha de 'lines'
    x = line.split()              # 'x' é uma lista de 'strings'
    if x[0] == 'MC' and x[1] == 'iteration':
        ener = float(x[5])       # converta 'string' pra 'float'
        energy.append(ener)     # energia: sexto elemento de 'x'

mean_e = np.mean(energy)                         # energia média
mean_e = round(mean_e, 4)           # média com 4 casas decimais
std_e = np.std(energy)                           # desvio padrão
std_e = round(std_e, 4)     # desvio padrão com 4 casas decimais

# linha a ser adicionada
newline = f'Average energy = {mean_e} +/- {std_e}\n'

with open('results.txt', 'a') as f:
    f.write(newline)      # linha é adicionada ao fim do arquivo

In [106]:
!more results.txt   # Forma de executar um comando do sistema operacional (Linux) no Jupyter-Notebook

MC iteration 1, energy = 3.4232 hartree
MC iteration 2, energy = 3.4783 hartree
MC iteration 3, energy = 3.4155 hartree
MC iteration 4, energy = 3.4369 hartree
MC simulation will proceed for 2 more steps
MC iteration 5, energy = 3.4423 hartree
The MC iterations took ca 5.4321 seconds
MC iteration 6, energy = 3.4510 hartree
Average energy = 3.4412 +/- 0.0203


## Ajustando o centro de massa de coordenadas cartesianas

In [107]:
%%file traj.xyz
2
time = 10 fs
H   -0.163148 +0.232363 +0.156896 
H   +0.163148 -0.232363 -0.156896 
2
time = 20 fs
H   +0.246005 +0.254485 +0.000000 
H   -0.246005 -0.254485 +0.000000 
2
time = 30 fs
H   -0.276528 -0.049833 +0.194428 
H   +0.276528 +0.049833 -0.194428

Overwriting traj.xyz


### Caixa de código 7: Colocando centros de massa de snapshots na origem (partes 1 e 2)

In [108]:
###### PARTE 1 ######
with open('traj.xyz') as f:
    lines = f.readlines()     # Salve todas as linhas em 'lines'

line1 = lines[0].split() # primeira linha de traj.xyz (= string)
n_atoms = int(line1[0])      # número total de átomos do sistema
lines_snap = n_atoms + 2         # número de linhas por snapshot
n_lines = len(lines)        # número total de linhas de traj.xyz
n_snapshots = n_lines // lines_snap        # número de snapshots

###### PARTE 2 ######
cm = []           # lista vazia para guardar os centros de massa
for snapshot in range(n_snapshots):
    XCM, YCM, ZCM = 0, 0, 0      # inicializar o centro de massa
    for i in range(n_atoms):
        index = snapshot * lines_snap + i + 2  # índice variável
        line = lines[index].split() # ['símbolo', 'X', 'Y', 'Z']
        x = float(line[1])               # coordenada X do átomo
        y = float(line[2])               # coordenada Y do átomo
        z = float(line[3])               # coordenada Z do átomo
        XCM += x                 # adicione x ao centro de massa
        YCM += y                 # adicione y ao centro de massa
        ZCM += z                 # adicione z ao centro de massa
    XCM, YCM, ZCM = XCM/n_atoms, YCM/n_atoms, ZCM/n_atoms
    cm.append([XCM, YCM, ZCM])         # guarde o CM do snapshot

### Caixa de código 8: Colocando centros de massa de *snapshots* na origem (parte 3)

In [109]:
###### PARTE 3 ######    
with open('traj_CM.xyz', 'w') as f:   # crie e edite traj_CM.xyz
    for snapshot in range(n_snapshots):    # snapshot = 0, 1 e 2
        L0 = lines[lines_snap * snapshot]     # número de átomos 
        f.write(L0)
        L1 = lines[lines_snap * snapshot + 1]       # comentário
        f.write(L1)
        for i in range(n_atoms):
            index = snapshot * lines_snap + i + 2 
            line = lines[index].split()
            atom = line[0] + ' '            # atom = tipo string
            x =  float(line[1]) - cm[snapshot][0]  # x corrigido
            y =  float(line[2]) - cm[snapshot][1]  # y corrigido
            z =  float(line[3]) - cm[snapshot][2]  # z corrigido
            x, y, z = str(x) + ' ', str(y) + ' ', str(z) + '\n'
            line = atom + x + y + z
            f.write(line)

In [110]:
!more traj_CM.xyz   # Forma de executar um comando do sistema operacional (Linux) no Jupyter-Notebook

2
time = 10 fs
H -0.163148 0.232363 0.156896
H 0.163148 -0.232363 -0.156896
2
time = 20 fs
H 0.246005 0.254485 0.0
H -0.246005 -0.254485 0.0
2
time = 30 fs
H -0.276528 -0.049833 0.194428
H 0.276528 0.049833 -0.194428


In [111]:
###### PARTE 3 ######   (alterada para melhor formatação) 
with open('traj_CM.xyz', 'w') as f:   # crie e edite traj_CM.xyz
    for snapshot in range(n_snapshots):    # snapshot = 0, 1 e 2
        L0 = lines[lines_snap * snapshot]     # número de átomos 
        f.write(L0)
        L1 = lines[lines_snap * snapshot + 1]       # comentário
        f.write(L1)
        for i in range(n_atoms):
            index = snapshot * lines_snap + i + 2 
            line = lines[index].split()
            atom = line[0] + ' '            # atom = tipo string
            x =  float(line[1]) - cm[snapshot][0]  # x corrigido
            y =  float(line[2]) - cm[snapshot][1]  # y corrigido
            z =  float(line[3]) - cm[snapshot][2]  # z corrigido
            line = f'{atom:<3} {x:+10.6f} {y:+10.6f} {z:+10.6f} \n'            
            f.write(line)

In [112]:
!more traj_CM.xyz   # Forma de executar um comando do sistema operacional (Linux) no Jupyter-Notebook

2
time = 10 fs
H    -0.163148  +0.232363  +0.156896 
H    +0.163148  -0.232363  -0.156896 
2
time = 20 fs
H    +0.246005  +0.254485  +0.000000 
H    -0.246005  -0.254485  +0.000000 
2
time = 30 fs
H    -0.276528  -0.049833  +0.194428 
H    +0.276528  +0.049833  -0.194428 


### Caixa de código 9: Executando os códigos anteriores com outros arquivos

In [114]:
#import sys
#filename = sys.argv[1]
filename = 'traj.xyz'

with open(filename) as f:
    lines = f.readlines()     # Salve todas as linhas em 'lines'
line1 = lines[0].split() # primeira linha de traj.xyz (= string)
n_atoms = int(line1[0])      # número total de átomos do sistema
lines_snap = n_atoms + 2         # número de linhas por snapshot
n_lines = len(lines)        # número total de linhas de traj.xyz
n_snapshots = n_lines // lines_snap        # número de snapshots

cm = []           # lista vazia para guardar os centros de massa
for snapshot in range(n_snapshots):
    XCM, YCM, ZCM = 0, 0, 0      # inicializar o centro de massa
    for i in range(n_atoms):
        index = snapshot * lines_snap + i + 2  # índice variável
        line = lines[index].split() # ['símbolo', 'X', 'Y', 'Z']
        x = float(line[1])               # coordenada X do átomo
        y = float(line[2])               # coordenada Y do átomo
        z = float(line[3])               # coordenada Z do átomo
        XCM += x                 # adicione x ao centro de massa
        YCM += y                 # adicione y ao centro de massa
        ZCM += z                 # adicione z ao centro de massa
    XCM, YCM, ZCM = XCM/n_atoms, YCM/n_atoms, ZCM/n_atoms
    cm.append([XCM, YCM, ZCM])         # guarde o CM do snapshot

with open(f'CM_{filename}', 'w') as f: # crie e edite CM_{?}.xyz
    for snapshot in range(n_snapshots):    # snapshot = 0, 1 e 2
        L0 = lines[lines_snap * snapshot]     # número de átomos 
        f.write(L0)
        L1 = lines[lines_snap * snapshot + 1]       # comentário
        f.write(L1)
        for i in range(n_atoms):
            index = snapshot * lines_snap + i + 2 
            line = lines[index].split()
            atom = line[0] + ' '            # atom = tipo string
            x =  float(line[1]) - cm[snapshot][0]  # x corrigido
            y =  float(line[2]) - cm[snapshot][1]  # y corrigido
            z =  float(line[3]) - cm[snapshot][2]  # z corrigido
            x, y, z = str(x) + ' ', str(y) + ' ', str(z) + '\n'
            line = atom + x + y + z
            f.write(line)

In [115]:
!more traj_CM.xyz   # Forma de executar um comando do sistema operacional (Linux) no Jupyter-Notebook

2
time = 10 fs
H    -0.163148  +0.232363  +0.156896 
H    +0.163148  -0.232363  -0.156896 
2
time = 20 fs
H    +0.246005  +0.254485  +0.000000 
H    -0.246005  -0.254485  +0.000000 
2
time = 30 fs
H    -0.276528  -0.049833  +0.194428 
H    +0.276528  +0.049833  -0.194428 
